### Scoring using the best model locally - Optional

The training environment in AML compute cluster can be different than the one locally. Hence the create a new Conda environment in the repo so that model can be loaded locally. 

    conda env create -f conda_env.yml

Use the model file downloaded using automl_amlsdkv2_training.ipynb. 

Load the model

In [ ]:
import joblib
import os

model_path = os.path.join("./artifact_downloads/outputs", "mlflow-model" ,"model.pkl")
model = joblib.load(model_path)

In [ ]:
model

Use the test dataset created in automl_amlsdkv2_training.ipynb

Format the test dataset and drop the Churn column. 

In [ ]:
import pandas as pd

test_data_orig = pd.read_csv("./wa_telco_customer_churn_test_data/WA_Fn-UseC_-Telco-Customer-Churn_Test.csv")

test_data = test_data_orig.drop("Churn", axis=1)
test_data["Partner"] = test_data["Partner"].map({"Yes": True, "No": False})
test_data["Dependents"] = test_data["Dependents"].map({"Yes": True, "No": False})
test_data["PhoneService"] = test_data["PhoneService"].map({"Yes": True, "No": False})
test_data["PaperlessBilling"] = test_data["PaperlessBilling"].map({"Yes": True, "No": False})
 


Run the prediction

In [ ]:

result = model.predict(test_data)

Merge prediction results with test data to compare the results. 

In [ ]:
test_data_orig["Churn_Prediction"] = result

test_data_orig['Churn_Prediction'] = test_data_orig['Churn_Prediction'].map({True: 'Yes', False: 'No'})


In [ ]:
test_data_orig.columns

In [ ]:
selected_columns = test_data_orig[['customerID', 'Churn', 'Churn_Prediction']]

# Display the selected columns
print(selected_columns)

Calculate accuracy, precision, recall, and F1-score

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming you have a DataFrame called 'test_data_orig'
# with 'Churn' representing actual labels ('No' or 'Yes') and 'Churn_Prediction' representing predicted labels ('No' or 'Yes')
actual_labels = (test_data_orig['Churn'] == 'Yes').astype(int)  # Convert 'Yes' to 1 and 'No' to 0
predicted_labels = (test_data_orig['Churn_Prediction'] == 'Yes').astype(int)  # Convert 'Yes' to 1 and 'No' to 0

# Calculate accuracy, precision, recall, and F1-score
accuracy = accuracy_score(actual_labels, predicted_labels)
precision = precision_score(actual_labels, predicted_labels)
recall = recall_score(actual_labels, predicted_labels)
f1 = f1_score(actual_labels, predicted_labels)

# Print the metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")


Accuracy (0.80): An accuracy of 80% indicates that your model correctly predicts whether a customer will churn or not in 80% of the cases. This is a reasonably good starting point, but it's important to consider other metrics due to the potential class imbalance in customer churn problems. For example, if only a small percentage of customers actually churn, a model that predicts "no churn" for all customers could still achieve a high accuracy, but it would not be useful.

Precision (0.65): Precision of 65% means that when your model predicts a customer will churn, it is correct 65% of the time. This metric is important because you want to avoid incorrectly flagging customers who don't intend to churn as potential churners. A precision of 65% suggests that the model is decent at identifying actual churners without too many false positives.

Recall (0.51): Recall of 51% means that your model is able to capture 51% of the actual churn cases. While this isn't extremely high, it's still meaningful because it indicates that the model can identify more than half of the customers who are likely to churn. Recall is important when you want to minimize false negatives and ensure you don't miss potential churners.

F1-Score (0.57): The F1-Score of 0.57 is a balanced metric that takes both precision and recall into account. It suggests that the model strikes a balance between identifying actual churners and avoiding false alarms.

In customer churn prediction, it's often more important to have a higher recall (to capture as many potential churners as possible) while maintaining a reasonable precision (to avoid unnecessarily alarming non-churners). However, the balance between precision and recall depends on the specific business goals and costs associated with churn.

Consider monitoring these metrics over time, and if possible, compare your model's performance to previous methods or industry benchmarks to assess whether further improvements are needed. Additionally, you might want to perform more advanced analyses, such as feature importance assessment or model tuning, to enhance your churn prediction model.

### Score using Compute Cluster in AML - Optional 

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

#### Provide values for your AML workspace

    subscription_id = "YOUR_SUBSCRIPTION_ID"
    resource_group = "YOUR_RESOURCE_GROUP"
    workspace = "YOUR_WORKSPACE_NAME"

In [ ]:

ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your Azure Machine Learning workspace
    subscription_id = "f1a8fafd-a8a3-46d8-bb5e-01deb63d275d"
    resource_group = "aml-rg"
    workspace = "aml-testws"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

Set the model name created in automl_amlsdkv2_training.ipynb

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import ModelType

model_name = "wa_telco_customer_churn_model_best"
model_versions = ml_client.models.list(name=model_name)
latest_model = max(model_versions, key=lambda x: x.version)
latest_model.path

In [ ]:
from azure.ai.ml import command, Input, Output
from azure.ai.ml.constants import AssetTypes, InputOutputModes
data_type = AssetTypes.MLTABLE
input_mode = InputOutputModes.RO_MOUNT
output_mode = InputOutputModes.RW_MOUNT

Create Input schema for scoring job

In [ ]:

#data_path = "./wa_telco_customer_churn_test_data_uri_file.csv"
inputs = {
    "input_data": Input(type=AssetTypes.URI_FILE, path=data_asset.path),
    "input_model": Input(type=AssetTypes.MLFLOW_MODEL, path=latest_model.path)
}

In [ ]:
inputs

Create Output schema for scoring job

In [ ]:
dataset_path = "wa-telco-customer-churn-predictions"
dataset_version = "1.0"
outputs = {
    "output_folder": Output(
        type=AssetTypes.URI_FOLDER,
        path=f"azureml://subscriptions/{subscription_id}/resourcegroups/{resource_group}/workspaces/{workspace}/datastores/workspaceblobstore/paths/{dataset_path}",
        name = dataset_path,
        version = dataset_version
    )
}

Define scoring job using Command component

Use the same environment as the model was created in AutoML. 
    
        AzureML-AutoML:142

Please review score.py

In [ ]:
from azure.ai.ml import command, Input


# define the command
command_job = command(
    code="./",
    command="python score.py --input_model ${{inputs.input_model}} --input_data ${{inputs.input_data}} --output_folder ${{outputs.output_folder}}",
    experiment_name="wa-telco-customer-churn-prediction",
    inputs=inputs,
    outputs=outputs,    
    environment="AzureML-AutoML:142",
    compute="cpu-cluster",
)

# Submit the command
ml_client.jobs.create_or_update(command_job)

Please review the results of the Scoring job in Aml Workspace. 